# Configuration Section - Important

In [23]:
colab_flag = False  # Set it true if the notebook is run on Colab
language_model ="fin_lab_large"  # imdb | abc | fin_unlabelled | fin_lab_balance | fin_lab_large
set_labels = "large"
pd.set_option('display.max_rows', 500)

# Importing Modules

## General Libraries

In [3]:
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm

## For Colab

In [4]:
if colab_flag:
  
  !pip install -Uqq fastbook   
  from fastbook import * 
  from google.colab import drive 
  drive.mount('/content/drive')
  %cd /content/drive/My\ Drive/Colab\ Notebooks/

  root = "/content/drive/My Drive/Colab Notebooks/"

  gpu_info = !nvidia-smi
  gpu_info = '\n'.join(gpu_info)
  if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
  else:
    print(gpu_info)

else:

  root = "C:/Users/user/ML/Ironhack/GitHub/Final_Project/"

## Deep Learning

In [5]:
from fastai.text.all import *

# Loading Data

In [6]:
with open(f"Data/dls/dls_clas_{set_labels}", "rb") as f:
    
    dls_clas = pickle.load(f)

    f.close()

    inferer = f"{language_model}_{set_labels}_inferer"

# Loading the LSTM model

In [7]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, metrics=accuracy).to_fp16()

C:\Users\ehern\miniconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [8]:
learn.load(inferer)

# Predicting the training/test dataset

In [10]:
fin_lab_large_df = pd.read_csv("Data/csv/fin_lab_large.csv")
fin_lab_large_df["prediction"]=fin_lab_large_df["prob_pos"] = fin_lab_large_df["prob_neg"] = fin_lab_large_df["certainty"] = None
fin_lab_large_df

,text,sentiment,set,prediction,prob_pos,prob_neg,certainty
0,Operating cash flow after investments totalled EUR 61.3 mn compared to EUR 75.3 mn in the corresponding period in 2007 .,negative,train,None,None,None,None
1,"TietoEnator was down 1.13 pct to 18.38 , extending recent lows after last week 's second-quarter report , dealers said .",negative,train,None,None,None,None
2,"In Finland 's Hobby Hall 's sales decreased by 10 % , and international sales fell by 19 % .",negative,train,None,None,None,None
3,"Svyturys-Utenos Alus , which is controlled by the Nordic group Baltic Beverages Holding ( BBH ) , posted a 4.7-per-cent growth in beer sales for January-May to 46.22 million litres .",positive,train,None,None,None,None
4,"`` Lining stone sales were also good in the early autumn , and order books are strong to the end of the year .",positive,train,None,None,None,None
...,...,...,...,...,...,...,...
1940,"The chain posted sales of 298 million euros for full 2005 , a rise of 19.5 percent , year-on-year .",positive,test,None,None,None,None
1941,Satama earned Data Management Solutions competency with Business Intelligence specialization recentlyvia the acquisition of Fimentor Oy .,positive,test,None,None,None,None
1942,"Rinkuskiai raised the sales by 18.1 percent , to 1.37 million liters , while the sales of Kauno Alus grew by 14.3 percent , to 960,000 liters .",positive,test,None,None,None,None
1943,"Earnings per share ( EPS ) in the first half of 2007 amounted to EUR0 .29 , down from EUR0 .40 year ago .",negative,test,None,None,None,None


In [11]:
pred = learn.predict(fin_lab_large_df["text"][0])
pred

C:\Users\ehern\miniconda3\lib\site-packages\torch\cuda\amp\grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


('negative', TensorText(0), TensorText([0.5896, 0.4104]))

In [12]:
for row in tqdm(range(0, len(fin_lab_large_df))):

  pred = learn.predict(fin_lab_large_df["text"][row])
  fin_lab_large_df["prediction"][row] = pred[0]
  fin_lab_large_df["prob_pos"][row] = float(pred[2][1])
  fin_lab_large_df["prob_neg"][row] = float(pred[2][0])
  fin_lab_large_df["certainty"][row] = abs(fin_lab_large_df["prob_pos"][row]-fin_lab_large_df["prob_neg"][row])

  0%|          | 0/1945 [00:00<?, ?it/s]C:\Users\ehern\miniconda3\lib\site-packages\torch\cuda\amp\grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


  0%|          | 1/1945 [00:00<07:09,  4.52it/s]C:\Users\ehern\miniconda3\lib\site-packages\torch\cuda\amp\grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


  0%|          | 2/1945 [00:00<06:43,  4.82it/s]C:\Users\ehern\miniconda3\lib\site-packages\torch\cuda\amp\grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


  0%|          | 3/1945 [00:00<06:29,  4.98it/s]C:\Users\ehern\miniconda3\lib\site-packages\torch\cuda\amp\grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


  0%|          | 4/1945 [00:00<07:08,  4.53it/s]C:\Users\ehern\miniconda3\lib\site-packages\torch\cuda\amp\grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


...

100%|█████████▉| 1941/1945 [09:45<00:01,  3.80it/s]C:\Users\ehern\miniconda3\lib\site-packages\torch\cuda\amp\grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


100%|█████████▉| 1942/1945 [09:46<00:00,  3.86it/s]C:\Users\ehern\miniconda3\lib\site-packages\torch\cuda\amp\grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


100%|█████████▉| 1943/1945 [09:46<00:00,  3.89it/s]C:\Users\ehern\miniconda3\lib\site-packages\torch\cuda\amp\grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


100%|█████████▉| 1944/1945 [09:46<00:00,  4.05it/s]C:\Users\ehern\miniconda3\lib\site-packages\torch\cuda\amp\grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


100%|██████████| 1945/1945 [09:46<00:00,  3.31it/s]


In [13]:
fin_lab_large_df

,text,sentiment,set,prediction,prob_pos,prob_neg,certainty
0,Operating cash flow after investments totalled EUR 61.3 mn compared to EUR 75.3 mn in the corresponding period in 2007 .,negative,train,negative,0.410379,0.589621,0.179242
1,"TietoEnator was down 1.13 pct to 18.38 , extending recent lows after last week 's second-quarter report , dealers said .",negative,train,negative,0.004138,0.995862,0.991723
2,"In Finland 's Hobby Hall 's sales decreased by 10 % , and international sales fell by 19 % .",negative,train,negative,0.007289,0.992711,0.985422
3,"Svyturys-Utenos Alus , which is controlled by the Nordic group Baltic Beverages Holding ( BBH ) , posted a 4.7-per-cent growth in beer sales for January-May to 46.22 million litres .",positive,train,positive,0.990804,0.009196,0.981608
4,"`` Lining stone sales were also good in the early autumn , and order books are strong to the end of the year .",positive,train,positive,0.995378,0.004622,0.990756
...,...,...,...,...,...,...,...
1940,"The chain posted sales of 298 million euros for full 2005 , a rise of 19.5 percent , year-on-year .",positive,test,positive,0.994779,0.005221,0.989558
1941,Satama earned Data Management Solutions competency with Business Intelligence specialization recentlyvia the acquisition of Fimentor Oy .,positive,test,positive,0.991149,0.008851,0.982298
1942,"Rinkuskiai raised the sales by 18.1 percent , to 1.37 million liters , while the sales of Kauno Alus grew by 14.3 percent , to 960,000 liters .",positive,test,positive,0.996699,0.003301,0.993398
1943,"Earnings per share ( EPS ) in the first half of 2007 amounted to EUR0 .29 , down from EUR0 .40 year ago .",negative,test,negative,0.01583,0.98417,0.96834


In [14]:
fin_lab_large_df.to_csv("Data/csv/fin_lab_large_pred.csv", index=False)


In [25]:
# Let see on wich ones the model was more certain

fin_lab_large_df.sort_values(by='certainty', ascending=True).head(250)

,text,sentiment,set,prediction,prob_pos,prob_neg,certainty
1381,"`` This is super nasty , '' said Eric Schultze , chief technology officer at Shavlik Technologies LLC , a St. Paul , Minn.-based maker of patch management tools .",negative,train,positive,0.503658,0.496342,0.007317
1187,Operating profit excluding non-recurring items increased by 27 % to EUR 81.9 mn from EUR 64.4 mn in the corresponding period in 2008 .,positive,train,positive,0.503737,0.496263,0.007475
323,"The antibody , given at repeated doses of up to 8 mg-kg , was generally well tolerated , and the pharmacokinetic characteristics of BTT-1023 in psoriasis patients were consistent with those observed in a previously completed study in rheumatoid arthritis RA patients .",positive,train,positive,0.507778,0.492222,0.015556
513,"Finnish Sampo Bank , of Danish Danske Bank group , reports profit before taxes of EUR 152.3 mn in 2010 , up from EUR 32.7 mn in 2009 .",positive,train,positive,0.512823,0.487177,0.025646
1689,"Pretax profit totalled EUR 2.0 mn , compared to a loss of EUR 159.2 mn in the fourth quarter of 2008 .",positive,test,positive,0.51461,0.48539,0.02922
1250,Operating profit was EUR 1.6 mn in 2005 compared to EUR 5.9 mn in 2004 .,negative,train,positive,0.515684,0.484316,0.031369
43,"Insurer Axa ( PAR : CS ) slid by 5.35 % to EUR 14.15 , after Citigroup and ING slashed their targets on the stock .",negative,train,negative,0.483034,0.516966,0.033933
514,"HELSINKI Thomson Financial - Shares closed little changed , with Cargotec and Huhtamaki dropping sharply on disappointing second-quarter reports .",negative,train,negative,0.466701,0.533299,0.066598
732,"Consolidated operating profit excluding one-off items was EUR 30.6 mn , up from EUR 29.6 mn a year earlier .",positive,train,positive,0.534351,0.465649,0.068701
844,"( ADP News ) - Oct 31 , 2008 - Finnish food company Raisio Oyj ( OMX : RAIVV ) said today that its net profit jumped to EUR 16.4 million ( USD 20.9 m ) for the first nine months of 2008 from EUR 1.1 million for the same period of 2",positive,train,negative,0.464121,0.535879,0.071758


In [26]:
# Let see on wich ones the model has failed

fin_lab_large_df[fin_lab_large_df["sentiment"] != fin_lab_large_df["prediction"]]

,text,sentiment,set,prediction,prob_pos,prob_neg,certainty
15,One of the challenges in the oil production in the North Sea is scale formation that can plug pipelines and halt production .,negative,train,positive,0.897245,0.102755,0.794491
25,"Frost sold shares for $ 19 million at $ 6.06-7 .12 per share , compared with Friday 's high of $ 11.33 and low of $ 10.14 .",negative,train,positive,0.992114,0.007886,0.984229
148,Finnish business software group AffectoGenimap Oyj said its net profit halved to 1.2 mln euro ( $ 1.5 mln ) in the first nine months of 2006 from 2.2 mln euro ( $ 2.8 mln ) in the same period of 2005 .,negative,train,positive,0.553155,0.446845,0.10631
272,"As a result , the Russia 's import restrictions on Finnish dairy companies will be canceled on 6 August 2010 .",positive,train,negative,0.058413,0.941587,0.883174
331,"According to the CEO of Nordea Bank Estonia Vahur Kraft , Nordea Finland and Stockmann have been cooperating for more than ten years .",positive,train,negative,0.357248,0.642752,0.285503
364,"`` However , the rapidly increasing costs and the strengthening of the euro were challenging for the whole industry , and they impacted on our results . ''",negative,train,positive,0.909394,0.090606,0.818788
372,The District Court of Helsinki dismissed all charges as the court of first instance in 2005 .,positive,train,negative,0.23326,0.76674,0.53348
397,"The company 's profit totaled Ls 578,100 in H1 2007 , down 30.9 % year-on-year .",negative,train,positive,0.577685,0.422315,0.155369
416,"Myllykoski , with one paper plant in Finland , one in the US and three in Germany , had revenues of EUR286m in the first half of 2010 and an operating loss of EUR12m , Reuters said .",negative,train,positive,0.571223,0.428777,0.142445
421,"Currently , the company foresees its pre-tax profit to remain below the 2009 level when it reached EUR 700,000 USD 934,000 , compared with previous projections of a slightly better pre-tax profit .",negative,train,positive,0.780616,0.219384,0.561233


# Using the model in new data


In [ ]:
guardian_df = pd.read_csv("Data/csv/guardian_headlines.csv")
guardian_df

,Time,Headlines
0,18-Jul-20,Johnson is asking Santa for a Christmas recovery
1,18-Jul-20,‘I now fear the worst’: four grim tales of working life upended by Covid-19
2,18-Jul-20,Five key areas Sunak must tackle to serve up economic recovery
3,18-Jul-20,Covid-19 leaves firms ‘fatally ill-prepared’ for no-deal Brexit
4,18-Jul-20,The Week in Patriarchy \n\n\n Bacardi's 'lady vodka': the latest in a long line of depressing gendered products
...,...,...
17795,17-Dec-17,How investing in solar energy can create a brighter future for Africa
17796,17-Dec-17,Poundland suppliers hit by insurance downgrade
17797,17-Dec-17,Cryptocurrencies: City watchdog to investigate new fundraising trend
17798,17-Dec-17,Unilever sells household name spreads to KKR for £6bn


In [ ]:
guardian_df = (pd.DataFrame(guardian_df["Headlines"]).rename({"Headlines": "text"}, axis=1)
                 .dropna()
                 .reset_index(drop=True) 
               )
guardian_df["prob_pos"] = guardian_df["prob_neg"] = guardian_df["certainty"] = guardian_df["sentiment"] = None
guardian_df

,text,prob_pos,prob_neg,certainty,sentiment
0,Johnson is asking Santa for a Christmas recovery,None,None,None,None
1,‘I now fear the worst’: four grim tales of working life upended by Covid-19,None,None,None,None
2,Five key areas Sunak must tackle to serve up economic recovery,None,None,None,None
3,Covid-19 leaves firms ‘fatally ill-prepared’ for no-deal Brexit,None,None,None,None
4,The Week in Patriarchy \n\n\n Bacardi's 'lady vodka': the latest in a long line of depressing gendered products,None,None,None,None
...,...,...,...,...,...
17795,How investing in solar energy can create a brighter future for Africa,None,None,None,None
17796,Poundland suppliers hit by insurance downgrade,None,None,None,None
17797,Cryptocurrencies: City watchdog to investigate new fundraising trend,None,None,None,None
17798,Unilever sells household name spreads to KKR for £6bn,None,None,None,None


In [ ]:
learn.predict(guardian_df["text"][0])

('negative', TensorText(0), TensorText([0.5471, 0.4529]))

In [ ]:
for row in tqdm(range(5000, 10000)):

  pred = learn.predict(guardian_df["text"][row])
  guardian_df["sentiment"][row] = pred[0]
  guardian_df["prob_pos"][row] = float(pred[2][1])
  guardian_df["prob_neg"][row] = float(pred[2][0])
  guardian_df["certainty"][row] = abs(guardian_df["prob_pos"][row]-guardian_df["prob_neg"][row])

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 3/5000 [00:00<03:20, 24.95it/s]

  0%|          | 6/5000 [00:00<03:30, 23.77it/s]

  0%|          | 9/5000 [00:00<03:22, 24.69it/s]

  0%|          | 12/5000 [00:00<03:24, 24.42it/s]

...

100%|█████████▉| 4989/5000 [03:23<00:00, 25.52it/s]

100%|█████████▉| 4992/5000 [03:23<00:00, 25.45it/s]

100%|█████████▉| 4995/5000 [03:23<00:00, 25.57it/s]

100%|█████████▉| 4998/5000 [03:24<00:00, 23.24it/s]

100%|██████████| 5000/5000 [03:24<00:00, 24.49it/s]


In [ ]:
for row in tqdm(range(10000, len(guardian_df))):

  pred = learn.predict(guardian_df["text"][row])
  guardian_df["sentiment"][row] = pred[0]
  guardian_df["prob_pos"][row] = float(pred[2][1])
  guardian_df["prob_neg"][row] = float(pred[2][0])
  guardian_df["certainty"][row] = abs(guardian_df["prob_pos"][row]-guardian_df["prob_neg"][row])

  0%|          | 0/7800 [00:00<?, ?it/s]

  0%|          | 3/7800 [00:00<05:13, 24.89it/s]

  0%|          | 6/7800 [00:00<05:41, 22.85it/s]

  0%|          | 9/7800 [00:00<05:32, 23.42it/s]

  0%|          | 12/7800 [00:00<05:34, 23.29it/s]

...

100%|█████████▉| 7788/7800 [05:12<00:00, 24.09it/s]

100%|█████████▉| 7791/7800 [05:12<00:00, 24.41it/s]

100%|█████████▉| 7794/7800 [05:12<00:00, 24.02it/s]

100%|█████████▉| 7797/7800 [05:12<00:00, 24.31it/s]

100%|██████████| 7800/7800 [05:12<00:00, 24.93it/s]


In [ ]:
guardian_df.to_csv("Data/csv/guardian_headlines_labeled.csv", index=False)

In [ ]:
guardian_df

,text,prob_pos,prob_neg,certainty,sentiment
0,Johnson is asking Santa for a Christmas recovery,0.452906,0.547094,0.0941872,negative
1,‘I now fear the worst’: four grim tales of working life upended by Covid-19,0.18279,0.81721,0.63442,negative
2,Five key areas Sunak must tackle to serve up economic recovery,0.650689,0.349311,0.301379,positive
3,Covid-19 leaves firms ‘fatally ill-prepared’ for no-deal Brexit,0.634331,0.365669,0.268662,positive
4,The Week in Patriarchy \n\n\n Bacardi's 'lady vodka': the latest in a long line of depressing gendered products,0.974264,0.0257361,0.948528,positive
...,...,...,...,...,...
17795,How investing in solar energy can create a brighter future for Africa,0.987204,0.0127965,0.974407,positive
17796,Poundland suppliers hit by insurance downgrade,0.0681024,0.931898,0.863795,negative
17797,Cryptocurrencies: City watchdog to investigate new fundraising trend,0.964288,0.0357122,0.928576,positive
17798,Unilever sells household name spreads to KKR for £6bn,0.877687,0.122313,0.755373,positive
